# **Extração Simples de Comentários da App Store e Play Store**
Esse notebook trás uma forma simples de coleta de dados das principais lojas de aplicativos utilizando uma biblioteca de scrapp para dados da Google Play e uma API pública da Apple.

| Aplicativo | Play Store ID           | App Store ID |
| ---------- | ----------------------- | ------------ | 
| Claro      | com.nvt.cs              | 526829675    |
| TIM        | br.com.timbrasil.meutim | 668591218    |
| Vivo       | br.com.vivo             | 475915064    |



In [ ]:
import requests
import time
from google_play_scraper import Sort, reviews
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
spark = SparkSession.builder.appName("session").getOrCreate()
spark.conf.set("spark.sql.adaptive.enable", "true")

**Play Store**

In [16]:
qtd_comentarios = 5000

dados_android = []

apps = {
    "Claro": "com.nvt.cs",
    "TIM": "br.com.timbrasil.meutim",
    "Vivo": "br.com.vivo"
}

for app, app_id in apps.items():
    result, _ = reviews(
        app_id,
        lang='pt',
        country='br',
        sort=Sort.NEWEST,
        count=qtd_comentarios
    )
        
    for review in result:
        dados_android.append({
            "app": app,
            "system": "Android",
            "score": review.get("score", None),
            "content": review.get("content", None),
            "app_version": review.get("appVersion", None),
            "date": review.get("at", None),
            "reply_content": review.get("replyContent", None),
            "reply_date": review.get("repliedAt", None)
        })

comentarios_android = spark.createDataFrame(dados_android, schema=["app", "system", "score", "content",
                                                           "app_version", "date", "reply_content", "reply_date"])

comentarios_android.show(5)

+-----+------+--------------------+-------------------+--------------------+-------------------+-------------+----------+
|  app|system|               score|            content|         app_version|               date|reply_content|reply_date|
+-----+------+--------------------+-------------------+--------------------+-------------------+-------------+----------+
|Claro|18.7.0|           muito boa|2025-05-30 21:20:05|                NULL|               NULL|            3|   Android|
|Claro|18.7.0|           ótimo app|2025-05-30 21:19:34|                NULL|               NULL|            5|   Android|
|Claro|18.8.0|muito fácil de in...|2025-05-30 21:18:09|                NULL|               NULL|            5|   Android|
|Claro|18.6.0|atende as necessi...|2025-05-30 21:05:18|                NULL|               NULL|            5|   Android|
|Claro|18.7.0|Otimo app! (14/05...|2025-05-30 20:34:54|Olá, Elisabete! E...|2023-04-26 12:22:22|            5|   Android|
+-----+------+----------

**Dados App Store** \
Observação: A API pública permite coletar no máxinmo 10 páginas de comentários

In [7]:
page = 100
dados_ios = []

apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

for app, app_id in apps.items():
    for p in range(1, page + 1):  # Coletar da página 1 até o limite máximo
        url = f'https://itunes.apple.com/rss/customerreviews/page={p}/id={app_id}/sortby=mostrecent/json'
        
        try:
            response = requests.get(url)
            data = response.json()
        except Exception as e:
            print(f"Erro ao acessar página {p} do app {app}: {e}")
            break

        # Se não houver mais comentários, para o loop
        if "feed" not in data or "entry" not in data["feed"]:
            print(f"Sem mais comentários na página {p} para o app {app}.")
            break

        for entry in data["feed"]["entry"][1:]:  # Ignora o primeiro, que são infos do app
            dados_ios.append({
                "app": app,
                "system": "iOS",
                "score": entry['im:rating']['label'],
                "content": entry['content']['label'],
                "app_version": entry['im:version']['label']
            })

        if p % 10 == 0:
            print(f"Pausa após coletar {p} páginas do app {app}...")
            time.sleep(5)

comentarios_ios = spark.createDataFrame(dados_ios, schema=["app", "system", "score", "content", "app_version"])

comentarios_ios.show(5)

Sem mais comentários na página 5 para o app Claro.
Sem mais comentários na página 6 para o app TIM.
Pausa após coletar 10 páginas do app Vivo...
Erro ao acessar página 11 do app Vivo: Expecting value: line 1 column 1 (char 0)
+-----+-------+--------------------+-------+-----------+
|  app| system|               score|content|app_version|
+-----+-------+--------------------+-------+-----------+
|Claro|17.53.0|Tenho essa aplica...|      1|        iOS|
|Claro|17.48.0|Poluído, muito ru...|      1|        iOS|
|Claro|17.46.0|App is not at all...|      1|        iOS|
|Claro|17.27.0|Boa tarde, sou cl...|      4|        iOS|
|Claro|17.26.0|Ate hoje nao ha s...|      1|        iOS|
+-----+-------+--------------------+-------+-----------+
only showing top 5 rows



In [17]:
#Juntando as bases
comentarios = comentarios_android.unionByName(comentarios_ios, allowMissingColumns=True)
comentarios_android.count(), comentarios_ios.count(), comentarios.count()

(15000, 900, 15900)

In [18]:
comentarios.write.mode("overwrite").parquet("comentarios.parquet")